In [21]:
import math
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from vocab import tokens

In [ ]:
class GPT(nn.Module):
    def __init__(self, tokens_list):
        super().__init__()
        self.tokens_list    = tokens_list
        self.max_tokens     = 10
        self.context_length = 64
        self.vocab_size     = 200
        self.n_layers       = 2      # agora duas camadas
        self.n_heads        = 4      # agora quatro cabeças
        self.n_embd         = 16

        # Embeddings
        self.wte = nn.Embedding(self.vocab_size, self.n_embd)
        self.wpe = nn.Embedding(self.context_length, self.n_embd)

        # MLP
        self.fc1 = nn.Linear(self.n_embd, 2 * self.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.fc2 = nn.Linear(2 * self.n_embd, self.n_embd)

        # Normalização
        self.ln = nn.LayerNorm(self.n_embd)
        self.ln.weight.requires_grad = False
        self.ln.bias.requires_grad = False

        # Atenção
        self.qkv_proj = nn.Linear(self.n_embd, 3 * self.n_embd)
        self.out_proj = nn.Linear(self.n_embd, self.n_embd)
        self.head_dim = self.n_embd // self.n_heads

        # Máscara causal
        self.mask = torch.tril(torch.ones(self.context_length,
                                          self.context_length)
                               ).view(self.context_length,
                                      self.context_length)

        # Final head
        self.final_ln = nn.LayerNorm(self.n_embd)
        self.lm_head   = nn.Linear(self.n_embd, self.vocab_size)

        assert self.n_embd % self.n_heads == 0, "n_embd deve ser divisível por n_heads"

    def mlp(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x

    def layer_norm(self, x):
        return self.ln(x)

    def self_attention(self, x):
        # x: (T, C)
        T, C = x.size()
        qkv = self.qkv_proj(x)               # (T, 3*C)
        q, k, v = qkv.chunk(3, dim=1)        # cada um (T, C)

        # projeção multi-head
        q = q.view(T, self.n_heads, self.head_dim).transpose(0,1)  # (nh, T, hd)
        k = k.view(T, self.n_heads, self.head_dim).transpose(0,1)
        v = v.view(T, self.n_heads, self.head_dim).transpose(0,1)

        # produto escalar e escala
        att = (q @ k.transpose(-2,-1)) / math.sqrt(self.head_dim)   # (nh, T, T)

        # máscara causal
        att = att.masked_fill(self.mask[:T, :T] == 0, float('-inf'))

        # softmax
        att = torch.softmax(att, dim=-1)

        # aplicação sobre v
        y = att @ v  # (nh, T, hd)

        # concatena heads
        y = y.transpose(0,1).contiguous().view(T, C)  # (T, C)

        # projeção final
        return self.out_proj(y)

    def tokens_idx(self, tokens_chosen):
        self.tokens_vocab = {token: idx for idx, token in enumerate(tokens)}
        self.idx_to_token = {idx: token for token, idx in self.tokens_vocab.items()}

        # Pegar os índices correspondentes
        indices = [self.tokens_vocab[token] for token in tokens_chosen]

        # Converter para tensor, se quiser passar ao modelo
        self.indices_tensor = torch.tensor(indices, dtype=torch.long)

    def token_idx_target(self, token):
        

    def forward(self):
        # 1) lookup embeddings
        idx = self.indices_tensor         # (T,)
        T   = idx.size(0)
        tok_emb = self.wte(idx)           # (T, n_embd)
        pos_emb = self.wpe(torch.arange(T, device=idx.device))  # (T, n_embd)
        x = tok_emb + pos_emb             # (T, n_embd)

        # 2) aplica N camadas de (LN → Atenção → Residual → LN → MLP → Residual)
        for _ in range(self.n_layers):
            # pré-atenção
            x_norm = self.layer_norm(x)
            attn_out = self.self_attention(x_norm)
            x = x + attn_out

            # pré-MLP
            x_norm = self.layer_norm(x)
            mlp_out = self.mlp(x_norm)
            x = x + mlp_out

        return x[-1]  # vetor do último token (n_embd,)

    def predict_logits(self, x):
        x = self.final_ln(x)  # (n_embd,)
        return self.lm_head(x)  # (vocab_size,)

    def predict_next_token(self):
        self.tokens_idx(self.tokens_list)
        vec = self.forward()               # (n_embd,)
        logits = self.predict_logits(vec)  # (vocab_size,)
        probs = torch.softmax(logits, dim=-1)
        idx  = torch.multinomial(probs, num_samples=1).item()
        return self.idx_to_token[idx]

    def predict_all_sentence(self):
        for step in range(self.max_tokens):
            nt = self.predict_next_token()
            self.tokens_list.append(nt)
            print(f"Passo {step+1}: {' '.join(self.tokens_list)}")
        return self.tokens_list


In [23]:
tokens_list = ["o", "gato", "pequeno"]
model = GPT(tokens_list)

In [28]:
model.tokens_vocab

{'eu': 0,
 'você': 1,
 'ele': 2,
 'ela': 3,
 'nós': 4,
 'vocês': 5,
 'eles': 6,
 'elas': 7,
 'me': 8,
 'te': 9,
 'se': 10,
 'nos': 11,
 'vos': 12,
 'lhe': 13,
 'lhes': 14,
 'mim': 15,
 'ti': 16,
 'si': 17,
 'ser': 18,
 'estar': 19,
 'ter': 20,
 'haver': 21,
 'fazer': 22,
 'poder': 23,
 'dizer': 24,
 'ver': 25,
 'dar': 26,
 'saber': 27,
 'querer': 28,
 'chegar': 29,
 'passar': 30,
 'dever': 31,
 'ficar': 32,
 'deixar': 33,
 'pensar': 34,
 'vir': 35,
 'conhecer': 36,
 'casa': 37,
 'tempo': 38,
 'dia': 39,
 'mundo': 40,
 'homem': 41,
 'mulher': 42,
 'vida': 43,
 'mão': 44,
 'olho': 45,
 'palavra': 46,
 'caminho': 47,
 'gato': 48,
 'cachorro': 49,
 'carro': 50,
 'livro': 51,
 'porta': 52,
 'rua': 53,
 'trabalho': 154,
 'dinheiro': 55,
 'noite': 56,
 'bom': 57,
 'mau': 58,
 'feliz': 59,
 'triste': 60,
 'grande': 61,
 'pequeno': 62,
 'novo': 63,
 'velho': 64,
 'forte': 65,
 'fraco': 66,
 'belo': 67,
 'feio': 68,
 'inteligente': 69,
 'rápido': 70,
 'lento': 71,
 'em': 72,
 'de': 73,
 'com': 7

In [24]:
for name, param in model.named_parameters():
    print(f"Nome: {name}")
    print(f"Shape: {param.shape}")
    print(f"Valores: {param}\n")


Nome: wte.weight
Shape: torch.Size([200, 16])
Valores: Parameter containing:
tensor([[-1.9411e-01,  2.3504e-01, -4.9729e-01,  ..., -8.0626e-01,
          8.2647e-01,  3.5054e-01],
        [-1.0369e+00, -8.7673e-01,  5.5250e-03,  ..., -7.8883e-01,
          1.1758e-01, -9.1098e-03],
        [-3.4526e-01,  1.6308e-01,  6.6631e-01,  ..., -9.3561e-01,
          2.6943e-01, -1.2901e-03],
        ...,
        [-7.7874e-01, -5.6155e-01,  1.7890e-01,  ..., -9.3923e-01,
          1.6842e+00,  1.7876e+00],
        [ 1.3884e+00, -8.5188e-01, -9.9703e-01,  ...,  1.6953e+00,
         -3.8701e-01,  7.8332e-01],
        [-1.7553e-01,  1.9899e-01,  9.0632e-01,  ..., -1.7981e-01,
         -2.8316e+00, -1.1663e+00]], requires_grad=True)

Nome: wpe.weight
Shape: torch.Size([64, 16])
Valores: Parameter containing:
tensor([[ 2.1085,  1.0800, -0.4896,  ..., -0.4711,  0.5440, -0.7357],
        [-2.0055, -1.5082,  1.0819,  ...,  0.6395, -1.3682,  0.4735],
        [-1.2321, -0.6910, -0.0711,  ..., -0.1204,  0.

In [25]:
print(model)

GPT(
  (wte): Embedding(200, 16)
  (wpe): Embedding(64, 16)
  (fc1): Linear(in_features=16, out_features=32, bias=True)
  (gelu): GELU(approximate='tanh')
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (qkv_proj): Linear(in_features=16, out_features=48, bias=True)
  (out_proj): Linear(in_features=16, out_features=16, bias=True)
  (final_ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=16, out_features=200, bias=True)
)


In [26]:
model.predict_next_token()

'dizer'

In [27]:
model.predict_all_sentence()

Passo 1: o gato pequeno céu
Passo 2: o gato pequeno céu sem
Passo 3: o gato pequeno céu sem amar
Passo 4: o gato pequeno céu sem amar antes
Passo 5: o gato pequeno céu sem amar antes em
Passo 6: o gato pequeno céu sem amar antes em dez
Passo 7: o gato pequeno céu sem amar antes em dez vão
Passo 8: o gato pequeno céu sem amar antes em dez vão casa
Passo 9: o gato pequeno céu sem amar antes em dez vão casa não
Passo 10: o gato pequeno céu sem amar antes em dez vão casa não eu


['o',
 'gato',
 'pequeno',
 'céu',
 'sem',
 'amar',
 'antes',
 'em',
 'dez',
 'vão',
 'casa',
 'não',
 'eu']